In [2]:
# created on Dec 24, 2020
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [4]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [10]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets1.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "Jdlgv0zvXnaZr41TBUT1saWq1"
    consumer_secret = "f7ce3KjjhPV59EWvIqPyONuIPO3tnClZIaFHf3kN8jDQCvtZCU"
    access_token = "1250822026857758720-z5xhvg5ZN7GUkNEp8tmHBg4h5Tuv1h"
    access_token_secret = "ylDnOyWuxPrnhx2H0X8K4jfTnwBf8Kkain2jxirDNHyUt"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)
    stream.filter(track=['National Park'], is_async=True)

{'created_at': 'Thu Apr 23 03:31:52 +0000 2020', 'id': 1253164661253513218, 'id_str': '1253164661253513218', 'text': '@NCbae79 Wow , I hope and pray your strength to overcome that!', 'display_text_range': [9, 62], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1253164404172087297, 'in_reply_to_status_id_str': '1253164404172087297', 'in_reply_to_user_id': 1030790170503733249, 'in_reply_to_user_id_str': '1030790170503733249', 'in_reply_to_screen_name': 'NCbae79', 'user': {'id': 70834125, 'id_str': '70834125', 'name': 'The KING has spoken!🤴🏿', 'screen_name': 'kingmeeeech', 'location': 'Norfolk, VA', 'url': 'http://www.mykahlraphael.com', 'description': 'Just beware these tweets are my thoughts & my thoughts only! Plz follow @mykahlraphael for photography!', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 457, 'friends_count': 496, 'listed_count': 16, 'favourit

{'created_at': 'Thu Apr 23 03:31:53 +0000 2020', 'id': 1253164664218886144, 'id_str': '1253164664218886144', 'text': '@BarryOcho Grilled cheese and tomato basil soup', 'display_text_range': [11, 47], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1253163896489336832, 'in_reply_to_status_id_str': '1253163896489336832', 'in_reply_to_user_id': 385226805, 'in_reply_to_user_id_str': '385226805', 'in_reply_to_screen_name': 'BarryOcho', 'user': {'id': 612727803, 'id_str': '612727803', 'name': 'Allyn West', 'screen_name': 'allynwest', 'location': 'Houston, TX', 'url': 'http://onebreathhou.org', 'description': 'Words, etc. Now: @EnvDefenseFund, @OneBreathHOU. Ex: Houston Chronicle, Rice Design Alliance, Swamplot.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 3704, 'friends_count': 2927, 'listed_count': 74, 'favourites_count': 32486, 'statuses_count': 6428, 'crea

{'created_at': 'Thu Apr 23 03:31:53 +0000 2020', 'id': 1253164665217179649, 'id_str': '1253164665217179649', 'text': '😂😂😂😂😂', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2810435552, 'id_str': '2810435552', 'name': 'ひ', 'screen_name': 'Wilsooo4L', 'location': '713', 'url': None, 'description': '4L. 🎌 #BKM', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 545, 'friends_count': 304, 'listed_count': 16, 'favourites_count': 17421, 'statuses_count': 31704, 'created_at': 'Mon Sep 15 01:33:33 +0000 2014', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.t

{'created_at': 'Thu Apr 23 03:31:53 +0000 2020', 'id': 1253164666567745541, 'id_str': '1253164666567745541', 'text': 'Most thrilling sporting event I’ve ever witnessed in person', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 25035940, 'id_str': '25035940', 'name': 'Sir Nick', 'screen_name': 'perSONICk', 'location': 'Memphis- We grind here', 'url': 'http://twiends.com/personick', 'description': 'Husby to @roxstarRebecca #BRONY! Memphis alum for Christ with my best effort. I take thought-provoking pics and DJ. Go Grizz, Cubs, MEMPHIS Tigers #grizznation', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1814, 'friends_count': 4985, 'listed_count': 29, 'favourites_count': 51519, 'statuses_count': 57077, 'create

{'created_at': 'Thu Apr 23 03:31:54 +0000 2020', 'id': 1253164667716931585, 'id_str': '1253164667716931585', 'text': 'We got company coming overrrrr 🎶', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 3137426200, 'id_str': '3137426200', 'name': 'ASHLAI', 'screen_name': 'ashlaisabelle_', 'location': 'Uptown, Bronx ', 'url': None, 'description': 'my twitter doesn’t have the answers you’re looking for || howard university alumna', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1128, 'friends_count': 505, 'listed_count': 15, 'favourites_count': 13319, 'statuses_count': 12352, 'created_at': 'Sun Apr 05 15:37:49 +0000 2015', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_ena

...

{'created_at': 'Thu Apr 23 03:32:51 +0000 2020', 'id': 1253164907224354822, 'id_str': '1253164907224354822', 'text': 'Y’ALL. 😩', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 64352010, 'id_str': '64352010', 'name': 'Arionne Nettles', 'screen_name': 'ArionneNettles', 'location': 'Chicago, the City of WINd', 'url': 'http://www.arionne.com', 'description': 'Lecturer at Northwestern’s @medillschool and publisher of @medillchicago | @AP and @WBEZ alum | ❤ Project: @WeWriteChicago | FAMU + NU-made | #specialneeds mama', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 4965, 'friends_count': 4189, 'listed_count': 136, 'favourites_count': 4306, 'statuses_count': 86795, 'created_at': 'Mon Aug 10 06:56:32 +0000 2009', 

{'created_at': 'Thu Apr 23 03:32:51 +0000 2020', 'id': 1253164908314755077, 'id_str': '1253164908314755077', 'text': '@maddiejean613 @CrawfordCollins i think not', 'display_text_range': [32, 43], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1253164855059775501, 'in_reply_to_status_id_str': '1253164855059775501', 'in_reply_to_user_id': 977614387170201600, 'in_reply_to_user_id_str': '977614387170201600', 'in_reply_to_screen_name': 'maddiejean613', 'user': {'id': 2346090114, 'id_str': '2346090114', 'name': '𝐚𝐬𝐡𝐥𝐞𝐲', 'screen_name': 'crawfordsjonah', 'location': '⅗ ♡ katie ♡ jonah ♡ crawford ♡', 'url': 'https://m.youtube.com/channel/UCudnLFE1MTENyvMafGYxRlQ', 'description': '“𝘪𝘧 𝘺𝘰𝘶 𝘨𝘰𝘵 𝘯𝘰 𝘰𝘯𝘦 𝘦𝘭𝘴𝘦 𝘵𝘰 𝘴𝘮𝘪𝘭𝘦 𝘧𝘰𝘳 𝘴𝘮𝘪𝘭𝘦 𝘧𝘰𝘳 𝘮𝘦" ~ @jonahmarais ♡ FAN ACCOUNT ☾ 21', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 23172, 'friends_count': 3339, 'listed_count': 315, 'f

{'created_at': 'Thu Apr 23 03:32:51 +0000 2020', 'id': 1253164909178654720, 'id_str': '1253164909178654720', 'text': '@_justmateo_ Surprise surprise, Star Wars fans remain trash', 'display_text_range': [13, 59], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1253058590765711362, 'in_reply_to_status_id_str': '1253058590765711362', 'in_reply_to_user_id': 540807611, 'in_reply_to_user_id_str': '540807611', 'in_reply_to_screen_name': '_justmateo_', 'user': {'id': 703187761893801984, 'id_str': '703187761893801984', 'name': 'Traveling-Restricted Salesman', 'screen_name': 'OpenMothersMale', 'location': 'Henderson, NV', 'url': 'https://boardwalktimes.net/boardwalktimes-net-boardwalk-talk/home', 'description': 'Unprofessional Theme Park Twitter Guy', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 771, 'friends_count': 987, 'listed_count': 10, 'favourites_count': 542

{'created_at': 'Thu Apr 23 03:32:51 +0000 2020', 'id': 1253164910130888704, 'id_str': '1253164910130888704', 'text': 'No ones cause I’m ugly', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1217976765257330689, 'id_str': '1217976765257330689', 'name': 'Taylor Treat', 'screen_name': 'realtreetreat', 'location': 'Florida, USA', 'url': None, 'description': 'God is great, beer is good, and people are crazy', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 184, 'friends_count': 373, 'listed_count': 0, 'favourites_count': 838, 'statuses_count': 877, 'created_at': 'Fri Jan 17 01:07:57 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator'

{'created_at': 'Thu Apr 23 03:32:52 +0000 2020', 'id': 1253164911158595584, 'id_str': '1253164911158595584', 'text': '@yangbangh @deairaaadior Yassss 😍', 'display_text_range': [25, 33], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': 1253164014554755075, 'in_reply_to_status_id_str': '1253164014554755075', 'in_reply_to_user_id': 1165413428841897984, 'in_reply_to_user_id_str': '1165413428841897984', 'in_reply_to_screen_name': 'yangbangh', 'user': {'id': 1125257262182871040, 'id_str': '1125257262182871040', 'name': 'Jacqui 🥶💋😈', 'screen_name': 'chitownescutie', 'location': 'Chicago, IL  ✈️ Columbia, SC❤️', 'url': None, 'description': '🎶🎶//Zen 💜 is my ☮️ 🧘🏾\u200d♂️ //back up account👉🏾 @chijac3 🇯🇲🇺🇸✊🏾 1st lady of 👉🏾#ChillSquad 🥶 😝💋 #thinklikebink #freebink his gangsta boo ❤️💚💛🤞🏾', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 9920, 'friends_count': 9332, 'lis